In [1]:
!pip uninstall -qqy jupyterlab
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.


In [2]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

In [4]:
# Retry automatically if API quota limit is hit or if model needs to make multiple consecutive calls
# 429 -> Too many calls
# 503 -> Service unavailable

from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
    genai.models.Models.generate_content = retry.Retry(predicate = is_retriable)(genai.models.Models.generate_content)

In [5]:
%load_ext sql
%sql sqlite:///sample.db

In [6]:
%%sql

CREATE TABLE IF NOT EXISTS Products(
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name VARCHAR(255) NOT NULL,
    price DECIMAL(10,2) NOT NULL
);

CREATE TABLE IF NOT EXISTS Staff(
    staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL
);

CREATE TABLE IF NOT EXISTS Orders(
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_name VARCHAR(255) NOT NULL,
    staff_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    FOREIGN KEY(staff_id) REFERENCES Staff(staff_id),
    FOREIGN KEY(product_id) REFERENCES Products(product_id)
);



INSERT INTO Products (product_name, price) VALUES
('Laptop', 799.99),
('Keyboard', 129.99),
('Mouse', 29.99);


INSERT INTO Staff (first_name, last_name) VALUES
('Alice', 'Smith'),
('Bob', 'Johnson'),
('Charlie', 'Williams');

INSERT INTO Orders (customer_name, staff_id, product_id) VALUES 
('David Lee', 1, 1),
('Emily Chen', 2, 2),
('Frank Brown', 1, 3);

 * sqlite:///sample.db
Done.
Done.
Done.
3 rows affected.
3 rows affected.
3 rows affected.


[]

In [7]:
import sqlite3

db_file = 'sample.db'
db_conn = sqlite3.connect(db_file)

In [8]:
def list_tables() -> list[str]:
    print('-DB CALL: list_tables()')
    cursor = db_conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
    tables = cursor.fetchall()

    return [t[0] for t in tables]

list_tables()

-DB CALL: list_tables()


['Products', 'sqlite_sequence', 'Staff', 'Orders']

In [9]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    print(f'-DB CALL: describe_table({table_name})')
    cursor = db_conn.cursor()
    
    cursor.execute(f'PRAGMA table_info({table_name});')
    schema = cursor.fetchall()

    return [(col[1], col[2]) for col in schema]

describe_table('Products')

-DB CALL: describe_table(Products)


[('product_id', 'INTEGER'),
 ('product_name', 'VARCHAR(255)'),
 ('price', 'DECIMAL(10,2)')]

In [10]:
def execute_query(sql: str) -> list[list[str]]:
    print(f'-DB CALL: execute_query({sql})')
    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()

execute_query('SELECT * FROM Products')

-DB CALL: execute_query(SELECT * FROM Products)


[(1, 'Laptop', 799.99), (2, 'Keyboard', 129.99), (3, 'Mouse', 29.99)]

In [11]:
db_tools = [list_tables, describe_table, execute_query]

instruction = """
You are a helpful chatbot that can interact with a SQL database
for a computer store. You will take the users questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand the
schema, and execute_query to issue a SQL SELECT query.
"""

client = genai.Client(api_key = GOOGLE_API_KEY)

chat = client.chats.create(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(
        system_instruction = instruction,
        tools = db_tools,
    ),
)

In [12]:
resp = chat.send_message('What is the cheapest product?')

print(f'\n{resp.text}')

-DB CALL: execute_query(SELECT * FROM products ORDER BY price LIMIT 1)

The cheapest product is the Mouse, which costs $29.99.



In [13]:
chat = client.chats.create(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(
        system_instruction = instruction,
        tools = db_tools,
    ),
)

resp = chat.send_message('What products should Alice focus on selling to round out her portfolio? Explain why.')

print(f'\n{resp.text}')

-DB CALL: list_tables()
-DB CALL: describe_table(Products)
-DB CALL: describe_table(Staff)
-DB CALL: describe_table(Orders)
-DB CALL: execute_query(SELECT Products.product_name FROM Products LEFT JOIN Orders ON Products.product_id = Orders.product_id LEFT JOIN Staff ON Orders.staff_id = Staff.staff_id WHERE Staff.first_name = 'Alice' GROUP BY Products.product_name ORDER BY COUNT(Orders.order_id) ASC LIMIT 5)

Alice should focus on selling Laptops and Mouse. These are the 2 least sold products by Alice. By focusing on these products, Alice can potentially increase her overall sales and diversify her portfolio.



In [14]:
import textwrap

def print_chat_turns(chat):
    for event in chat.get_history():
        print(f'{event.role.capitalize()}:')
        for part in event.parts:
            if txt := part.text:
                print(f'"{txt}""')
            elif fn := part.function_call:
                args = ", ".join(f'{key} = {val}' for key, val in fn.args.items())
                print(f'Function call {fn.name}({args})')
            elif resp := part.function_response:
                print("Function response")
                print(textwrap.indent(str(resp.response['result']), "    "))
        print()

print_chat_turns(chat)

User:
"What products should Alice focus on selling to round out her portfolio? Explain why.""

Model:
Function call list_tables()

User:
Function response
    ['Products', 'sqlite_sequence', 'Staff', 'Orders']

Model:
Function call describe_table(table_name = Products)

User:
Function response
    [('product_id', 'INTEGER'), ('product_name', 'VARCHAR(255)'), ('price', 'DECIMAL(10,2)')]

Model:
Function call describe_table(table_name = Staff)

User:
Function response
    [('staff_id', 'INTEGER'), ('first_name', 'VARCHAR(255)'), ('last_name', 'VARCHAR(255)')]

Model:
Function call describe_table(table_name = Orders)

User:
Function response
    [('order_id', 'INTEGER'), ('customer_name', 'VARCHAR(255)'), ('staff_id', 'INTEGER'), ('product_id', 'INTEGER')]

Model:
Function call execute_query(sql = SELECT Products.product_name FROM Products LEFT JOIN Orders ON Products.product_id = Orders.product_id LEFT JOIN Staff ON Orders.staff_id = Staff.staff_id WHERE Staff.first_name = 'Alice' GROUP 